In [1]:
import pandas as pd
import numpy as np

In [2]:
def fix_spectrum(df):
    aux = df
    aux[spectrum_cols + ['edad', 'peso', 'talla', 'radiopezon']] = aux[spectrum_cols + ['edad', 'peso', 'talla', 'radiopezon']].applymap(lambda x: np.nan if type(x) == str and not x.startswith('0.') else x)
    return aux

def ubicacion_to_mama(y):
    if str(y)[0:2] in ['1-', '2-', '3-', '4-', '5-']:
        return 'left'
    elif str(y)[0] in ['6', '7', '8', '9', '10']:
        return 'right'
    else:
        return 'left_2'

In [3]:
dxa_df = (
    pd.read_csv('data_raw/ECO11200-ScioAnita_DATA_2020-11-25_1901.csv', sep=',')
        .loc[lambda x: x['check_scio'] == 1]
        .drop_duplicates('cod_child')
        .pipe(lambda x: x[['cod_child'] + [i for i in x.columns if i.startswith('dxa')]])
        .melt(id_vars='cod_child', var_name='medicion_dxa', value_name='valor_dxa')
        .assign(mama = lambda x: x['medicion_dxa'].map(lambda y: 'left' if '_lle' in y else 'right' if '_rle' in y else 'left_2'),
                valor_dxa = lambda x: x['valor_dxa'].abs())
)

In [4]:
scio_df = pd.read_csv('data_raw/Collection_21_nov_2019.csv', sep=',')
scio_cols = scio_df.columns
spectrum_cols = [i for i in scio_cols if i.startswith('spectrum')]
sample_cols = [i for i in scio_cols if i.startswith('sample')]

scio_df = (
    scio_df
        .drop(columns=[i for i in scio_cols if i.startswith('wr')])
        .pipe(fix_spectrum)
        .assign(**{col: (lambda c: lambda x: x[c].astype(float))(col) for col in spectrum_cols + sample_cols})
        .rename(columns={i: i[:-3] for i in spectrum_cols + sample_cols})
        .drop(columns=['v1019', 'espacio'])
        .assign(mama = lambda x: x['ubicacion'].map(ubicacion_to_mama),
                ubicacion = lambda x: x['ubicacion'].astype(str))
)

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (5,7,9,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
(
    scio_df
        .merge(dxa_df, left_on=['folio', 'mama'], right_on=['cod_child', 'mama'])
        .drop(columns=['cod_child', 'id']
        .to_csv('data/scio_dxa_full_27_11.csv')
)

In [5]:
# Densidad only
(
    scio_df
        .merge(dxa_df, left_on=['folio', 'mama'], right_on=['cod_child', 'mama'])
        .loc[lambda x: x['medicion_dxa'].str.contains('_bd')]
        .drop(columns=['cod_child', 'id'])
        .to_csv('data/scio_dxa_densidad_28_11.csv')
)